In [1]:
import numpy as np
import scipy
from scipy.sparse import diags
from scipy.linalg import hilbert
from scipy.sparse.linalg import cg
import time

In [2]:
def conjugate_gradient(A, b):
    x0 = np.ones(len(b))
    r0 = np.dot(A, x0) - b
    p_k = - r0
    r_k_r = np.dot(r0, r0)
    x_k1 = x0
    r_k1 = r0
    while True:
        Ap_k = np.dot(A, p_k)
        alpha_k = r_k_r / np.dot(p_k, Ap_k)
        x_k1 += alpha_k * p_k
        r_k1 += alpha_k * Ap_k
        r_k1_r = np.dot(r_k1, r_k1)
        if r_k1_r < 1e-8:
            break
        beta = r_k1_r / r_k_r
        r_k_r = r_k1_r
        p_k = beta * p_k - r_k1
    return x_k1

# Q10.a.

In [3]:
n_list = [100,400,1600]

A1_results = []
A2_results = []

for N in n_list:
    b = np.ones(N)
    A1 = diags([-1, -4, -1], [-1, 0, 1], shape=(N, N)).toarray()
    A2 = scipy.linalg.hilbert(N)

    t1 = time.time()
    x = conjugate_gradient(A1, b)
    t2 = time.time()
    A1_results.append([N, np.linalg.norm(np.dot(A1, x) - b),t2 - t1])

    t1 = time.time()
    x = conjugate_gradient(A2, b)
    t2 = time.time()
    A2_results.append([N, np.linalg.norm(np.dot(A2, x) - b),t2 - t1])

In [4]:
print("A1 results:\n")
print("{:<8} \t {:<8} \t\t {:<40}".format('N', '||A1x* - b||','Time'))
print("-"*65)
for v in A1_results:
    N,norm, t = v
    print ("{:<8} \t {:<8} \t {:<40}".format(N, norm, t))

A1 results:

N        	 ||A1x* - b|| 		 Time                                    
-----------------------------------------------------------------
100      	 9.201221302187598e-05 	 0.02588486671447754                     
400      	 9.25095388996844e-05 	 0.0009200572967529297                   
1600     	 9.261506119378308e-05 	 0.021431922912597656                    


In [5]:
print("A2 results:\n")
print("{:<8} \t {:<8} \t\t {:<40}".format('N', '||A2x* - b||','Time'))
print("-"*65)
for v in A2_results:
    N, norm, t = v
    print ("{:<8} \t {:<8} \t {:<40}".format(N, norm, t))

A2 results:

N        	 ||A2x* - b|| 		 Time                                    
-----------------------------------------------------------------
100      	 7.480837045165894e-05 	 0.01806807518005371                     
400      	 8.628143836240043e-05 	 0.013210773468017578                    
1600     	 9.038339898349864e-05 	 0.4534740447998047                      


In [6]:
A1 = hilbert(100)
b = np.ones(100)
x3 = cg(A1, b)
# print(x3[0])
print(np.linalg.norm(np.dot(A1, x3[0]) - b))
t4 = time.time()

6.220968269692673e-05


# Q10.b.

In [7]:
epsilon = 1e-6
n_list = [100,400,1600]

A1_results2 = []
A2_results2 = []

for N in n_list:
    b = np.ones(N)
    A1 = diags([-1, -4, -1], [-1, 0, 1], shape=(N, N)).toarray()
    A1_prime = A1 + epsilon
    A2 = scipy.linalg.hilbert(N)
    A2_prime = A2 + epsilon

    x = conjugate_gradient(A1, b)
    t1 = time.time()
    x_prime = conjugate_gradient(A1_prime, b)
    t2 = time.time()
    A1_results2.append([N, np.linalg.norm(np.dot(A1_prime, x_prime) - b), np.linalg.norm(A1 - A1_prime), 
                        np.linalg.norm(x - x_prime), t2 - t1])

    x = conjugate_gradient(A2, b)
    t1 = time.time()
    x_prime = conjugate_gradient(A2_prime, b)
    t2 = time.time()
    A2_results2.append([N, np.linalg.norm(np.dot(A2_prime, x_prime) - b), np.linalg.norm(A2 - A2_prime), 
                        np.linalg.norm(x - x_prime), t2 - t1])

In [8]:
print("A1' results:\n")
print("{} \t {:<16} \t\t {:<24} \t {:<24} \t {}".format('N', "||A1'x'* - b||","||A1 - A1'||","||x* - x'*||",'Time'))
print("-"*130)
for v in A1_results2:
    N, norm1, norm2, norm3, t = v
    print ("{} \t {:<16} \t\t {:<24} \t {:<24} \t {}".format(N, norm1, norm2, norm3, t))

A1' results:

N 	 ||A1'x'* - b||   		 ||A1 - A1'||             	 ||x* - x'*||             	 Time
----------------------------------------------------------------------------------------------------------------------------------
100 	 9.201375306615714e-05 		 0.00010000000000019656   	 2.8034778599532327e-05   	 0.026253223419189453
400 	 9.251571313054464e-05 		 0.0004000000000002083    	 0.00022274979090903437   	 0.0006122589111328125
1600 	 9.263977165815284e-05 		 0.0016000000000009056    	 0.001779277603018894     	 0.008115291595458984


In [9]:
print("A2' results:\n")
print("{} \t {:<16} \t\t {:<24} \t {:<24} \t {}".format('N', "||A2'x'* - b||","||A2 - A2'||","||x* - x'*||",'Time'))
print("-"*130)
for v in A2_results2:
    N, norm1, norm2, norm3, t = v
    print ("{} \t {:<16} \t\t {:<24} \t {:<24} \t {}".format(N, norm1, norm2, norm3, t))

A2' results:

N 	 ||A2'x'* - b||   		 ||A2 - A2'||             	 ||x* - x'*||             	 Time
----------------------------------------------------------------------------------------------------------------------------------
100 	 6.928157677985581e-05 		 9.999999999998317e-05    	 431.31513945127404       	 0.0010690689086914062
400 	 8.970951116915818e-05 		 0.00040000000000002046   	 6586.817477061563        	 0.010984182357788086
1600 	 5.910716676936034e-05 		 0.0016000000000007022    	 162170.23535708713       	 0.49959897994995117


# Q10.C.

When we add epsilon to A1, x* won't change a lot and ||x\* - x'\*|| is small.
<br/>
When we add epsilon to A2, x* won't change a lot and ||x\* - x'\*|| is big.
<br/>
In both of them ||A - A'|| and ||A'x\* - b|| is small i.e. A and A' are close and x'\* is a good answer to new problem but when A=A1 x\* and x'\* are close and when A=A2 they are far.